In [1]:
# !pip install line_profiler

In [2]:
from collections import deque
import numpy as np
import pandas as pd
import pandas_ta as ta

In [3]:
from  helpers_db import get_engine, run_sql
engine = get_engine()
with engine.begin() as conn:
  data = run_sql(conn, """SELECT * FROM ticks_1w WHERE symbol='PYPL' ORDER BY dt""")
  df = pd.DataFrame(data)
  df.set_index("dt", inplace = True)
# print(df)

In [54]:
def debug_deque(df, deq, key):
    vals = []
    for item in reversed(deq):
        vals.append(df.iloc[item][key])
    return vals

def clean_deque(i, k, deq, df, key, isHigh):
    if deq and deq[0] == i - k:
        deq.popleft()
    if isHigh:
        while deq and df.iloc[i][key] > df.iloc[deq[-1]][key]:
            deq.pop()
    else:
        while deq and df.iloc[i][key] < df.iloc[deq[-1]][key]:
            deq.pop()

def pivotPoints(pivot=None,data=None):
    """
    This function calculates the pivot points based on the pivot lenght.
    These can be HH, LH , HL, LL values based on the adjacent pivots 
    which occur based on the length of the pivot.
    """

    data['PH'] = False
    data['PHV'] = np.NaN
    data['PL'] = False
    data['PLV'] = np.NaN
    keyHigh = 'high'
    keyLow = 'low'
    win_size = pivot * 2 + 1
    deqHigh = deque()
    deqLow = deque()
    max_idx = 0
    min_idx = 0
    i = 0
    j = pivot
    pivot_low = None
    pivot_high = None
    for index, row in data.iterrows():
        if i in [10, 20, 30, 40, 50, 60]:
            print(i)

        if i < win_size:
            clean_deque(i, win_size, deqHigh, data, keyHigh, True)
            clean_deque(i, win_size, deqLow, data, keyLow, False)
            deqHigh.append(i)
            deqLow.append(i)
            if data.iloc[i][keyHigh] > data.iloc[max_idx][keyHigh]:
                max_idx = i
            if data.iloc[i][keyLow] < data.iloc[min_idx][keyLow]:
                min_idx = i
            if i == win_size-1:
                if data.iloc[max_idx][keyHigh] == data.iloc[j][keyHigh]:
                    data.at[data.index[j], 'PH'] = True
                    pivot_high = data.iloc[j][keyHigh]
                if data.iloc[min_idx][keyLow] == data.iloc[j][keyLow]:
                    data.at[data.index[j], 'PL'] = True
                    pivot_low = data.iloc[j][keyLow]
        if i >= win_size:
            j += 1
            clean_deque(i, win_size, deqHigh, data, keyHigh, True)
            clean_deque(i, win_size, deqLow, data, keyLow, False)
            deqHigh.append(i)
            deqLow.append(i)
            # pivot_val = data.iloc[deqHigh[0]][keyHigh]
            if data.iloc[deqHigh[0]][keyHigh] == data.iloc[j][keyHigh]:
                data.at[data.index[j], 'PH'] = True
                pivot_high = data.iloc[j][keyHigh]
            if data.iloc[deqLow[0]][keyLow] == data.iloc[j][keyLow]:
                data.at[data.index[j], 'PL'] = True
                pivot_low = data.iloc[j][keyLow]

        data.at[data.index[j], 'PHV'] = pivot_high
        data.at[data.index[j], 'PLV'] = pivot_low
        i = i + 1

    return data

df_pivots =  pivotPoints(pivot=11,data=df)
df['PH'] = df_pivots['PH']
df['PHV'] = df_pivots['PHV']
df['PL'] = df_pivots['PL']
df['PLV'] = df_pivots['PLV']

# df_pivots = df[(df.PH == True) | (df.PL == True)]

# df_pivots.sort_values(by=["dt"], ascending=False).head(20)
df_pivots.head(40)



KeyboardInterrupt: 

In [18]:
from typings import List
values = [42.55,39.91,39.69,39,38.52,37.32,36.95,36.65,36.17,35.91]
positions = [3,5,6,7,18,19,20,21,22,23]
val = 36.58
pos = 23

def lifo_highest_checker(
    values: List[float],
    # positions: List[int],
    value: float,
    # position: int,
) -> List[float|int]:
  while values[-1] < val:
    values.pop()
    # positions.pop()
  values.append(value)
  # positions.append(position)
  return values

lifo_highest_checker(values, positions, val, pos)


[42.55, 39.91, 39.69, 39, 38.52, 37.32, 36.95, 36.65, 36.58]

In [40]:
# from typings import list
highest = [40.2,42.55,39.2,39.91,39.69,39,35.46,35.51,34.95,34.06,34.69,33.68,33.02,35.05,35.75,36.65,38.52,37.32,36.95,36.65,36.17,35.91]
positions = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]
lowest = [35.5,36.91,36.22,38.43,37.01,33.83,30,33.41,32.61,32.55,33.1899,30.3,31.09,32.1,32.88,34.5,35.45,34.5,34.53,34.88,33.85,34.42]

# def filter_highest(items: [float], is_high: bool) -> [float]:
#   arr = []
#   for item in items:
#     if len(arr) > 0:
#       if is_high:
#         while len(arr) > 0 and item > arr[-1]:
#           arr.pop()
#       else:
#         while len(arr) > 0 and item < arr[-1]:
#           arr.pop()
#     arr.append(item)
#   return arr

# # filter_highest(highest, True)
# filter_highest(lowest, False)

def filter_hhll_positions(items: [float], positions: [int], is_high: bool) -> [int]:
  arr = []
  arr_positions = []
  for idx, item in enumerate(items):
    if len(arr) > 0:
      if is_high:
        while len(arr) > 0 and item > arr[-1]:
          arr.pop()
          arr_positions.pop()
      else:
        while len(arr) > 0 and item < arr[-1]:
          arr.pop()
          arr_positions.pop()
    arr.append(item)
    arr_positions.append(idx)
  return arr_positions

filter_hhll_positions(highest, positions, True)


[1, 3, 4, 5, 16, 17, 18, 19, 20, 21]